In [381]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
import selenium
import os
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import json
from pprint import pprint
import sqlalchemy
from sqlalchemy import create_engine,inspect, func, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn')
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

# Connect to chromedriver and BS

In [386]:
executable_path = {'executable_path': '/Users/hmm794/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'http://www.dph.illinois.gov/data-statistics/vital-statistics/illinois-population-data'
browser.visit(url)

# EXTRACT DATA

# Scrape Data

In [387]:
html = browser.html
soup = bs(html, 'html.parser')
soup.findAll('tr', limit=1)

[<tr><th>
 <p>County</p>
 </th>
 <th>April 1, 1980<br/>Census</th>
 <th>April 1, 1990<br/>Census</th>
 <th>April 1, 2000<br/>Census</th>
 <th>April 1, 2010<br/>Census</th>
 <th>Percent Change<br/>1980 to 1990</th>
 <th>Percent Change<br/>1990 to 2000</th>
 <th>Percent Change<br/>2000 to 2010</th>
 </tr>]

In [388]:
#test to see if the code works- 
soup.findAll('tr', limit=1)[0].findAll('th') 

[<th>
 <p>County</p>
 </th>,
 <th>April 1, 1980<br/>Census</th>,
 <th>April 1, 1990<br/>Census</th>,
 <th>April 1, 2000<br/>Census</th>,
 <th>April 1, 2010<br/>Census</th>,
 <th>Percent Change<br/>1980 to 1990</th>,
 <th>Percent Change<br/>1990 to 2000</th>,
 <th>Percent Change<br/>2000 to 2010</th>]

In [389]:
#get column headers
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=1)[0].findAll('th')]
column_headers

['\nCounty\n',
 'April 1, 1980Census',
 'April 1, 1990Census',
 'April 1, 2000Census',
 'April 1, 2010Census',
 'Percent Change1980 to 1990',
 'Percent Change1990 to 2000',
 'Percent Change2000 to 2010']

In [390]:
column_headers[column_headers.index('\nCounty\n')] = 'County'
print(column_headers)

['County', 'April 1, 1980Census', 'April 1, 1990Census', 'April 1, 2000Census', 'April 1, 2010Census', 'Percent Change1980 to 1990', 'Percent Change1990 to 2000', 'Percent Change2000 to 2010']


In [391]:
data_rows = soup.findAll('tr')[1:]  # skip the first  header rows

In [392]:
type(data_rows)  # now we have a list of table rows

list

In [393]:
census = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [394]:
df = pd.DataFrame(census,columns=column_headers)
df.head(5)

,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,Illinois,"\n11,426,518\n","\n11,430,602\n","\n12,419,293\n","\n12,830,632\n",\n0.04\n,\n8.6\n,\n3.3\n
1,Adams,"\n71,622\n","\n66,090\n","\n68,277\n","\n67,103\n",\n-7.7\n,\n3.3\n,\n-1.7\n
2,Alexander,"\n12,264\n","\n10,626\n","\n9,590\n","\n8,238\n",\n-13.4\n,\n-9.7\n,\n-14.1\n
3,Bond,"\n16,224\n","\n14,991\n","\n17,633\n","\n17,768\n",\n-7.6\n,\n17.6\n,\n0.8\n
4,Boone,"\n28,630\n","\n30,806\n","\n41,786\n","\n54,165\n",\n7.6\n,\n35.6\n,\n29.6\n


In [395]:
df.dtypes

County                        object
April 1, 1980Census           object
April 1, 1990Census           object
April 1, 2000Census           object
April 1, 2010Census           object
Percent Change1980 to 1990    object
Percent Change1990 to 2000    object
Percent Change2000 to 2010    object
dtype: object

# TRANSFORM

In [396]:
# df['April 1, 1980Census']= df['April 1, 1980Census'].astype(float)
df['April 1, 1980Census'] = df['April 1, 1980Census'].str.replace(r'\D', '')
df['April 1, 1990Census'] = df['April 1, 1990Census'].str.replace(r'\D', '')
df['April 1, 2000Census'] = df['April 1, 2000Census'].str.replace(r'\D', '')
df['April 1, 2010Census'] = df['April 1, 2010Census'].str.replace(r'\D', '')
df['Percent Change1980 to 1990'] = df['Percent Change1980 to 1990'].str.replace(r'\D', '')
df['Percent Change1990 to 2000'] = df['Percent Change1990 to 2000'].str.replace(r'\D', '')
df['Percent Change2000 to 2010'] = df['Percent Change2000 to 2010'].str.replace(r'\D', '')

In [397]:
df.head(5)

,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,Illinois,11426518,11430602,12419293,12830632,004,86,33
1,Adams,71622,66090,68277,67103,77,33,17
2,Alexander,12264,10626,9590,8238,134,97,141
3,Bond,16224,14991,17633,17768,76,176,08
4,Boone,28630,30806,41786,54165,76,356,296


In [398]:
f = df.apply(pd.to_numeric, errors='ignore')
f.dtypes

County                        object
April 1, 1980Census            int64
April 1, 1990Census            int64
April 1, 2000Census            int64
April 1, 2010Census            int64
Percent Change1980 to 1990     int64
Percent Change1990 to 2000     int64
Percent Change2000 to 2010     int64
dtype: object

In [399]:
f.reset_index(level=0, inplace=True)
f.head(5)

,index,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,0,Illinois,11426518,11430602,12419293,12830632,4,86,33
1,1,Adams,71622,66090,68277,67103,77,33,17
2,2,Alexander,12264,10626,9590,8238,134,97,141
3,3,Bond,16224,14991,17633,17768,76,176,8
4,4,Boone,28630,30806,41786,54165,76,356,296


In [400]:
illinois_pop = f[['index','County', 'April 1, 2010Census']]
illinois_pop = illinois_pop.rename(columns={"index":'id',"County":'county','April 1, 2010Census':'april_count'})
illinois_pop.head(10)

,id,county,april_count
0,0,Illinois,12830632
1,1,Adams,67103
2,2,Alexander,8238
3,3,Bond,17768
4,4,Boone,54165
5,5,Brown,6937
6,6,Bureau,34978
7,7,Calhoun,5089
8,8,Carroll,15387
9,9,Cass,13642


In [401]:
illinois_pop.dtypes

id              int64
county         object
april_count     int64
dtype: object

# extract data for second table

In [402]:
df2 = pd.read_csv("https://www.ers.usda.gov/webdocs/DataFiles/48747/Unemployment.csv?v=9115.7")

employ = df2[['FIPS', 'State', 'Area_name', 'Civilian_labor_force_2017', 'Employed_2017', 'Unemployed_2017',
       'Unemployment_rate_2017']]
employ.head(5)

,FIPS,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017
0,0,US,United States,"160,273,906","153,293,830","6,980,076",4.4
1,1000,AL,Alabama,2178243,2081676,96567,4.4
2,1001,AL,"Autauga County, AL",25972,24958,1014,3.9
3,1003,AL,"Baldwin County, AL",92090,88342,3748,4.1
4,1005,AL,"Barbour County, AL",8317,7831,486,5.8


# extract data for third table

In [403]:
df3 = pd.read_csv('https://www.ers.usda.gov/webdocs/DataFiles/48747/PovertyEstimates.csv?v=2529')

broke = df3[['FIPStxt', 'POVALL_2017', 'PCTPOVALL_2017', 'MEDHHINC_2017']]

poor = pov.rename(columns = {'FIPStxt': 'FIPS', 'POVALL_2017': 'Total_poverty_2017', 'PCTPOVALL_2017': 'Poverty_rate_2017',
                               'MEDHHINC_2017': 'Median_income_2017'})

poor.head(5)

,FIPS,Total_poverty_2017,Poverty_rate_2017,Median_income_2017
0,0,"42,583,651",13.4,"60,336"
1,1000,"802,263",16.9,"48,193"
2,1001,"7,390",13.4,"58,343"
3,1003,"21,199",10.1,"56,607"
4,1005,"7,414",33.4,"32,490"


# TRANSFORM DATA


# Clean up and merge tables

In [404]:
povertydf = pd.merge(employ, poor, how = 'inner', on = 'FIPS', validate = '1:1')
povertydf.head(5)

,FIPS,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Total_poverty_2017,Poverty_rate_2017,Median_income_2017
0,0,US,United States,"160,273,906","153,293,830","6,980,076",4.4,"42,583,651",13.4,"60,336"
1,1000,AL,Alabama,2178243,2081676,96567,4.4,"802,263",16.9,"48,193"
2,1001,AL,"Autauga County, AL",25972,24958,1014,3.9,"7,390",13.4,"58,343"
3,1003,AL,"Baldwin County, AL",92090,88342,3748,4.1,"21,199",10.1,"56,607"
4,1005,AL,"Barbour County, AL",8317,7831,486,5.8,"7,414",33.4,"32,490"


In [405]:
IL_poverty = povertydf.loc[povertydf['State'] == 'IL']
IL_poverty1 = IL_poverty[['FIPS','Area_name', 'Unemployment_rate_2017', 'Poverty_rate_2017']]
IL_poverty2 = IL_poverty1.rename(columns={"FIPS":'id',"Area_name":'county',"Unemployment_rate_2017":'unemployment',"Poverty_rate_2017":'poverty'})
IL_poverty2.head(5)

,id,county,unemployment,poverty
608,17000,Illinois,4.9,12.5
609,17001,"Adams County, IL",3.9,13.5
610,17003,"Alexander County, IL",8.4,30.3
611,17005,"Bond County, IL",4.5,13.3
612,17007,"Boone County, IL",6.8,8.8


In [406]:
IL_poverty2['unemployment'] = IL_poverty2['unemployment'].astype(int)
IL_poverty2['poverty'] = IL_poverty2['poverty'].astype(int)
IL_poverty2.dtypes

id               int64
county          object
unemployment     int64
poverty          int64
dtype: object

# export data as csv

In [278]:
poverty.to_csv('Poverty_data.csv', index=False, header=True)
illinois_pop.to_csv('Illinois_Census_2010_only.csv', index=True, header=True)
IL_poverty3.to_csv('Illinois_poverty_data_2017.csv', index=False, header=True)

# LOAD

# Create database

In [ ]:
pd.read_sql_query('SELECT authors FROM poetry.authors_df LIMIT 3', conn)

# Connect to PG Admin server

In [449]:
password = "Radeke23@"
rds_connection_string = f'postgres:{password}@localhost:5432/ETL-Project_MCS2019'
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.connect

In [455]:
engine.table_names()

['poverty', 'census']

### Use pandas to load csv converted DataFrame into database

In [451]:
IL_poverty2.to_sql(name='poverty', con=engine, if_exists='append', index=False)

In [452]:
illinois_pop.to_sql(name='census', con=engine, if_exists='append', index=False)

# Get a list of column names and types

In [458]:
columns = inspector.get_columns('illinois_poverty')
for c in columns:
    print(c['name'], c["type"])

id BIGINT
county TEXT
unemployment BIGINT
poverty BIGINT


In [459]:
columns = inspector.get_columns('illinois_census')
for c in columns:
    print(c['name'], c["type"])

id BIGINT
county TEXT
april_count BIGINT


In [478]:
# census = pd.read_sql_query('select * from illinois_census', con=engine)
# census.sort_values(by = 'April_count', ascending=False, inplace=True,)
# census.head(10)
county = pd.read_sql_query('SELECT * FROM census LIMIT 105', con=engine)
poverty = pd.read_sql_query('SELECT poverty FROM poverty LIMIT 105', con=engine)

In [479]:
county.head(10)

,id,county,april_count
0,0,Illinois,12830632
1,1,Adams,67103
2,2,Alexander,8238
3,3,Bond,17768
4,4,Boone,54165
5,5,Brown,6937
6,6,Bureau,34978
7,7,Calhoun,5089
8,8,Carroll,15387
9,9,Cass,13642


In [480]:
poverty.reset_index(inplace=True)
poverty1 = poverty.rename(columns={"index":'id'})
poverty1.head(10)

,id,poverty
0,0,12
1,1,13
2,2,30
3,3,13
4,4,8
5,5,14
6,6,10
7,7,10
8,8,11
9,9,13


In [488]:
finaldf = pd.merge(county, poverty1, how = 'inner', on = 'id', validate = '1:1')
finaldf.head(5)

,id,county,april_count,poverty
0,0,Illinois,12830632,12
1,1,Adams,67103,13
2,2,Alexander,8238,30
3,3,Bond,17768,13
4,4,Boone,54165,8


In [495]:
finaldf.sort_values(by = "poverty", ascending=False, inplace=True)
finaldf.head(100)

,id,county,april_count,poverty
2,2,Alexander,8238,30
39,39,Jackson,60218,29
77,77,Pulaski,6161,22
35,35,Hardin,4320,21
10,10,Champaign,201081,19
28,28,Franklin,39561,19
92,92,Vermilion,81625,19
91,91,Union,17808,19
15,15,Coles,53873,19
76,76,Pope,4470,18


In [501]:
finaldf['poverty'].describe()

count    103.000000
mean      12.873786
std        4.405165
min        4.000000
25%       10.000000
50%       13.000000
75%       15.000000
max       30.000000
Name: poverty, dtype: float64

In [503]:
finaldf['april_count'].describe()

count    1.030000e+02
mean     2.491385e+05
std      1.357361e+06
min      4.320000e+03
25%      1.469100e+04
50%      2.971800e+04
75%      5.633150e+04
max      1.283063e+07
Name: april_count, dtype: float64

In [502]:
cook_county = finaldf.loc[finaldf['county'] == 'Cook']
cook_county

,id,county,april_count,poverty
16,16,Cook,5194675,14


# Unemployment in Cook county is relatively better than most other counties in Illinois. The population falls around the 75th percentile however its unemployment rate is only in the 50th percentile. 